<a href="https://colab.research.google.com/github/jkjh23/BusanAIDeveloperCourse_AI_BaseballProject/blob/main/Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모듈 불러오기

In [ ]:
import sys

from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re

import pandas as pd
import warnings

In [ ]:
warnings.filterwarnings("ignore")

chrome_options = wd.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = wd.Chrome('./tool/chromedriver.exe', chrome_options=chrome_options)
driver.implicitly_wait(3)

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


# 네이버 - 월별 경기 일정

## 크롤링

In [ ]:
years = [i for i in range(2015,2021)]
months = [j for j in range(3, 12)]

pre_df = list()

for year in years :
    for month in months :
        target_site = f'https://sports.news.naver.com/kbaseball/schedule/index.nhn?date=20201025&month={month}&year={year}&teamCode='
        driver.get(target_site)

        selectorList = list()
        for i in range(len(driver.find_elements_by_css_selector('#calendarWrap > div > table > tbody > tr > td'))) :
            selectorList.append(driver.find_elements_by_css_selector('#calendarWrap > div > table > tbody > tr > td')[i].text)

        try:
            element=WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, 'sch_tb')))

            selectorListComplete = list()
            dateIdx = list()
            for idx, lst in enumerate(selectorList):
                if ' (' in lst :
                    dateIdx.append(idx)

            for i in range(0, len(dateIdx)) :
                if i < len(dateIdx)-1 :
                    selectorListTime = selectorList[dateIdx[i]+1 : dateIdx[i+1]]
                else :
                    selectorListTime = selectorList[dateIdx[i]+1 : ]

                timeIdx = list()
                for idx, lst in enumerate(selectorListTime):
                    if ':00' in lst :
                        timeIdx.append(idx)
                    elif ':30' in lst :
                        timeIdx.append(idx)

                selectorListTimeComplete = list()
                for j in range(0, len(timeIdx))  :
                    if j < len(timeIdx)-1 :
                        selectorListTimeComplete.append(selectorListTime[timeIdx[j] : timeIdx[j+1]])
                    else :
                        selectorListTimeComplete.append(selectorListTime[timeIdx[j] : ])

                selectorListComplete.append([selectorList[dateIdx[i]], selectorListTimeComplete])

            for i in range (0, len(selectorListComplete)) :
                if len(selectorListComplete[i][1]) != 0 :
                    for j in range(0, len(selectorListComplete[i][1])) :
                        pre_df.append({
                            "연도"     : year,
                            "월"       : selectorListComplete[i][0].split(".")[0],
                            "일"       : selectorListComplete[i][0].split(".")[1].split()[0],
                            "요일"     : selectorListComplete[i][0].split(".")[1].split()[1][1],
                            "시각"     : selectorListComplete[i][1][j][0],
                            '원정팀'   : selectorListComplete[i][1][j][1].split()[0],
                            '홈팀'     : selectorListComplete[i][1][j][1].split()[2], 
                            '점수'     : selectorListComplete[i][1][j][1].split()[1],
                            '경기장'   : selectorListComplete[i][1][j][-2]
                        }) 

        except Exception as e:
            print('오류 발생', e, year, month)

In [ ]:
naverSchedule = pd.DataFrame(pre_df)
naverSchedule

,연도,월,일,요일,시각,원정팀,홈팀,점수,경기장
0,2015,3,7,토,13:00,KIA,NC,1:2,마산
1,2015,3,7,토,13:00,kt,넥센,0:5,목동
2,2015,3,7,토,13:00,LG,한화,3:9,대전
3,2015,3,7,토,13:00,두산,삼성,9:4,포항
4,2015,3,7,토,13:00,SK,롯데,2:1,사직
...,...,...,...,...,...,...,...,...,...
5267,2020,10,30,금,18:30,KT,한화,VS,대전
5268,2020,10,30,금,18:30,LG,SK,VS,문학
5269,2020,10,30,금,18:30,NC,삼성,VS,대구
5270,2020,10,30,금,18:30,키움,두산,VS,잠실


## 쓸모없는 데이터 제거

In [ ]:
naverSchedule = naverSchedule.drop_duplicates()
naverSchedule

,연도,월,일,요일,시각,원정팀,홈팀,점수,경기장
0,2015,3,7,토,13:00,KIA,NC,1:2,마산
1,2015,3,7,토,13:00,kt,넥센,0:5,목동
2,2015,3,7,토,13:00,LG,한화,3:9,대전
3,2015,3,7,토,13:00,두산,삼성,9:4,포항
4,2015,3,7,토,13:00,SK,롯데,2:1,사직
...,...,...,...,...,...,...,...,...,...
5152,2020,10,30,금,18:30,KT,한화,VS,대전
5153,2020,10,30,금,18:30,LG,SK,VS,문학
5154,2020,10,30,금,18:30,NC,삼성,VS,대구
5155,2020,10,30,금,18:30,키움,두산,VS,잠실


In [ ]:
dropIdxList = naverSchedule[(naverSchedule['원정팀'] == '드림') | (naverSchedule['원정팀'] == '나눔')].index
naverSchedule = naverSchedule.drop(dropIdxList)
naverSchedule['원정팀'].unique()

array(['KIA', 'kt', 'LG', '두산', 'SK', '한화', '롯데', 'NC', '삼성', '넥센', 'KT',
       '키움'], dtype=object)

In [ ]:
dropIdxList = naverSchedule[naverSchedule['홈팀'] == '경기취소'].index
naverSchedule = naverSchedule.drop(dropIdxList)
naverSchedule['홈팀'].unique()

array(['NC', '넥센', '한화', '삼성', '롯데', 'KIA', 'kt', '두산', 'LG', 'SK', 'KT',
       '키움'], dtype=object)

In [ ]:
dropIdxList = naverSchedule[naverSchedule['점수'] == 'VS'].index
naverSchedule = naverSchedule.drop(dropIdxList)
naverSchedule['점수'].unique()

array(['1:2', '0:5', '3:9', '9:4', '2:1', '4:0', '4:10', '3:2', '0:9',
       '1:9', '6:3', '1:0', '2:0', '8:4', '5:2', '6:5', '10:7', '3:3',
       '5:6', '2:3', '0:12', '3:4', '0:1', '11:5', '5:3', '4:11', '11:1',
       '6:4', '9:3', '4:5', '5:4', '7:2', '3:5', '10:8', '5:5', '10:2',
       '12:0', '2:2', '2:4', '13:2', '4:3', '4:13', '1:1', '8:2', '7:10',
       '9:12', '1:3', '4:9', '1:6', '6:7', '1:4', '7:3', '7:1', '8:6',
       '3:0', '5:1', '3:10', '6:11', '5:0', '3:14', '10:4', '2:9', '4:1',
       '4:16', '13:7', '17:4', '13:5', '2:13', '4:2', '9:10', '0:6',
       '2:5', '9:6', '9:7', '11:8', '3:15', '18:2', '7:6', '8:3', '5:10',
       '0:10', '6:1', '1:12', '6:10', '4:8', '5:7', '6:8', '15:4', '12:9',
       '0:2', '6:2', '7:5', '14:4', '3:7', '3:11', '9:2', '1:7', '11:4',
       '2:6', '7:4', '6:0', '5:8', '6:9', '4:12', '3:1', '12:2', '11:2',
       '8:15', '8:5', '13:4', '10:6', '0:3', '3:6', '11:6', '8:9', '2:10',
       '0:0', '10:5', '11:10', '10:1', '4:6', '10:12',

## 팀 이름 통합

In [ ]:
def teamName(x) : 
  if x == '넥센' : return '키움'
  elif x == 'kt'   : return 'KT'
  else             : return x

naverSchedule['원정팀'] = naverSchedule['원정팀'].apply(teamName)
naverSchedule['홈팀'] = naverSchedule['홈팀'].apply(teamName)
naverSchedule.reset_index(drop=True, inplace=True)
naverSchedule

,연도,월,일,요일,시각,원정팀,홈팀,점수,경기장
0,2015,3,7,토,13:00,KIA,NC,1:2,마산
1,2015,3,7,토,13:00,KT,키움,0:5,목동
2,2015,3,7,토,13:00,LG,한화,3:9,대전
3,2015,3,7,토,13:00,두산,삼성,9:4,포항
4,2015,3,7,토,13:00,SK,롯데,2:1,사직
...,...,...,...,...,...,...,...,...,...
4645,2020,10,24,토,17:00,LG,NC,3:3,창원
4646,2020,10,24,토,17:00,롯데,두산,1:2,잠실
4647,2020,10,24,토,17:00,삼성,KIA,11:7,광주
4648,2020,10,25,일,14:00,롯데,KT,5:10,수원


## 점수 및 승패

In [ ]:
naverSchedule['원정팀점수'] = naverSchedule['점수'].apply(lambda x : x.split(':')[0])
naverSchedule['홈팀점수'] = naverSchedule['점수'].apply(lambda x : x.split(':')[1])
naverSchedule

,연도,월,일,요일,시각,원정팀,홈팀,점수,경기장,원정팀점수,홈팀점수
0,2015,3,7,토,13:00,KIA,NC,1:2,마산,1,2
1,2015,3,7,토,13:00,KT,키움,0:5,목동,0,5
2,2015,3,7,토,13:00,LG,한화,3:9,대전,3,9
3,2015,3,7,토,13:00,두산,삼성,9:4,포항,9,4
4,2015,3,7,토,13:00,SK,롯데,2:1,사직,2,1
...,...,...,...,...,...,...,...,...,...,...,...
4645,2020,10,24,토,17:00,LG,NC,3:3,창원,3,3
4646,2020,10,24,토,17:00,롯데,두산,1:2,잠실,1,2
4647,2020,10,24,토,17:00,삼성,KIA,11:7,광주,11,7
4648,2020,10,25,일,14:00,롯데,KT,5:10,수원,5,10


In [ ]:
warnings.filterwarnings("ignore")

naverSchedule['원정팀승리'] = [1 if int(naverSchedule['원정팀점수'][i]) > int(naverSchedule['홈팀점수'][i]) else 0 for i in range(0, naverSchedule.shape[0])]
naverSchedule['홈팀승리'] = [1 if int(naverSchedule['홈팀점수'][i]) > int(naverSchedule['원정팀점수'][i]) else 0 for i in range(0, naverSchedule.shape[0])]

naverSchedule

,연도,월,일,요일,시각,원정팀,홈팀,점수,경기장,원정팀점수,홈팀점수,원정팀승리,홈팀승리
0,2015,3,7,토,13:00,KIA,NC,1:2,마산,1,2,0,1
1,2015,3,7,토,13:00,KT,키움,0:5,목동,0,5,0,1
2,2015,3,7,토,13:00,LG,한화,3:9,대전,3,9,0,1
3,2015,3,7,토,13:00,두산,삼성,9:4,포항,9,4,1,0
4,2015,3,7,토,13:00,SK,롯데,2:1,사직,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4645,2020,10,24,토,17:00,LG,NC,3:3,창원,3,3,0,0
4646,2020,10,24,토,17:00,롯데,두산,1:2,잠실,1,2,0,1
4647,2020,10,24,토,17:00,삼성,KIA,11:7,광주,11,7,1,0
4648,2020,10,25,일,14:00,롯데,KT,5:10,수원,5,10,0,1


## 타입 변경

In [ ]:
naverSchedule['월'] = naverSchedule['월'].apply(int)
naverSchedule['일'] = naverSchedule['일'].apply(int)
naverSchedule['원정팀점수'] = naverSchedule['원정팀점수'].apply(int)
naverSchedule['홈팀점수'] = naverSchedule['홈팀점수'].apply(int)
naverSchedule['원정팀승리'] = naverSchedule['원정팀승리'].apply(int)
naverSchedule['홈팀승리'] = naverSchedule['홈팀승리'].apply(int)
naverSchedule.dtypes

연도        int64
월         int64
일         int64
요일       object
시각       object
원정팀      object
홈팀       object
점수       object
경기장      object
원정팀점수     int64
홈팀점수      int64
원정팀승리     int64
홈팀승리      int64
dtype: object

## 팀별 기록으로 분할

In [ ]:
naverSchedule.columns

Index(['연도', '월', '일', '요일', '시각', '원정팀', '홈팀', '점수', '경기장', '원정팀점수', '홈팀점수',
       '원정팀승리', '홈팀승리'],
      dtype='object')

In [ ]:
awayDf = naverSchedule.loc[:,['연도', '월', '일', '요일', '시각', '원정팀', '원정팀점수', '원정팀승리']]
awayDf.columns = ['연도', '월', '일', '요일', '시각', '팀명', '득점', '승리']
awayDf['홈'] = 0
awayDf

,연도,월,일,요일,시각,팀명,득점,승리,홈
0,2015,3,7,토,13:00,KIA,1,0,0
1,2015,3,7,토,13:00,KT,0,0,0
2,2015,3,7,토,13:00,LG,3,0,0
3,2015,3,7,토,13:00,두산,9,1,0
4,2015,3,7,토,13:00,SK,2,1,0
...,...,...,...,...,...,...,...,...,...
4645,2020,10,24,토,17:00,LG,3,0,0
4646,2020,10,24,토,17:00,롯데,1,0,0
4647,2020,10,24,토,17:00,삼성,11,1,0
4648,2020,10,25,일,14:00,롯데,5,0,0


In [ ]:
homeDf = naverSchedule.loc[:,['연도', '월', '일', '요일', '시각', '홈팀', '홈팀점수', '홈팀승리']]
homeDf.columns = ['연도', '월', '일', '요일', '시각', '팀명', '득점', '승리']
homeDf['홈'] = 1
homeDf

,연도,월,일,요일,시각,팀명,득점,승리,홈
0,2015,3,7,토,13:00,NC,2,1,1
1,2015,3,7,토,13:00,키움,5,1,1
2,2015,3,7,토,13:00,한화,9,1,1
3,2015,3,7,토,13:00,삼성,4,0,1
4,2015,3,7,토,13:00,롯데,1,0,1
...,...,...,...,...,...,...,...,...,...
4645,2020,10,24,토,17:00,NC,3,0,1
4646,2020,10,24,토,17:00,두산,2,1,1
4647,2020,10,24,토,17:00,KIA,7,0,1
4648,2020,10,25,일,14:00,KT,10,1,1


In [ ]:
naverDf = pd.concat([awayDf, homeDf])
naverDf

,연도,월,일,요일,시각,팀명,득점,승리,홈
0,2015,3,7,토,13:00,KIA,1,0,0
1,2015,3,7,토,13:00,KT,0,0,0
2,2015,3,7,토,13:00,LG,3,0,0
3,2015,3,7,토,13:00,두산,9,1,0
4,2015,3,7,토,13:00,SK,2,1,0
...,...,...,...,...,...,...,...,...,...
4645,2020,10,24,토,17:00,NC,3,0,1
4646,2020,10,24,토,17:00,두산,2,1,1
4647,2020,10,24,토,17:00,KIA,7,0,1
4648,2020,10,25,일,14:00,KT,10,1,1


In [ ]:
naverDf.sort_values(by=['연도', '월', '일'], axis=0, inplace=True)
naverDf.reset_index(drop=True, inplace=True)
naverDf

,연도,월,일,요일,시각,팀명,득점,승리,홈
0,2015,3,7,토,13:00,KIA,1,0,0
1,2015,3,7,토,13:00,KT,0,0,0
2,2015,3,7,토,13:00,LG,3,0,0
3,2015,3,7,토,13:00,두산,9,1,0
4,2015,3,7,토,13:00,SK,2,1,0
...,...,...,...,...,...,...,...,...,...
9295,2020,10,24,토,17:00,KIA,7,0,1
9296,2020,10,25,일,14:00,롯데,5,0,0
9297,2020,10,25,일,14:00,삼성,1,0,0
9298,2020,10,25,일,14:00,KT,10,1,1


In [ ]:
naverDf.to_csv('naver_schedule.csv', encoding='cp949')

# KBreport - 일별 경기 기록

## 투수 기록

In [ ]:
pre_df_team_p = list()
target_year_ls = ['2020']
target_month_ls = ['07', '08', '09', '10', '11']
target_day_ls = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']

In [ ]:
pre_df_team_p = list()

for target_year in target_year_ls :
    for target_month in target_month_ls :
        for target_day in target_day_ls :
            target_site = f'http://www.kbreport.com/teams/pitcher/main?rows=20&order=TPCT&orderType=DESC&teamId=&pitcher_type=&year_from=2015&year_to=2020&split01=day&split02_1={target_year}-{target_month}-{target_day}&split02_2={target_year}-{target_month}-{target_day}'
            driver.get(target_site)

            try:
                element=WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ltb-table')))

                results = list()
                for td in driver.find_elements_by_css_selector('#resultListDiv > table > tbody > tr') :
                    results.append(td.text.split())

                if len(results) > 2 :
                    for i in range (1, len(results)-1) :
                            pre_df_team_p.append({
                                "연도" : target_year,
                                "월"   : target_month,
                                "일"   : target_day,
                                "팀명" : results[i][1],
                                '팀승' : results[i][3], 
                                '팀무' : results[i][4], 
                                '팀패' : results[i][5], 
                                'RA/G' : results[i][8], 
                                '실점' : results[i][9], 
                                '자책' : results[i][10], 
                                '피홈런' : results[i][11], 
                                '볼넷%' : results[i][12], 
                                '삼진%' : results[i][13], 
                                'BABIP' : results[i][14], 
                                'LOB%' : results[i][15], 
                                'ERA' : results[i][16], 
                                'RA9-WAR' : results[i][17], 
                                'FIP' : results[i][18],
                                'kFIP' : results[i][19],
                                'WAR' : results[i][20]
                            }) 

            except Exception as e: #내가 찾고자하는게 안올라올 수도 있으니까
                print('오류 발생', e)

In [ ]:
pitDf = pd.DataFrame(pre_df_team_p)

In [ ]:
pitDf.to_csv('kbreport_pit_main_day_200701_201025.csv', encoding='cp949')

## 타자 기록

In [ ]:
target_year_ls = ['2020']
target_month_ls = ['07', '08', '09', '10', '11']
target_day_ls = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']

pre_df_team_h = list()

for target_year in target_year_ls :
    for target_month in target_month_ls :
        for target_day in target_day_ls :
            target_site = f'http://www.kbreport.com/teams/main?rows=20&order=TPCT&orderType=DESC&teamId=&defense_no=&year_from=2015&year_to=2020&split01=day&split02_1={target_year}-{target_month}-{target_day}&split02_2={target_year}-{target_month}-{target_day}'

            driver.get(target_site)

            try:
                element=WebDriverWait(driver,60).until(EC.presence_of_element_located((By.CLASS_NAME, 'ltb-table')))

                results = list()
                for td in driver.find_elements_by_css_selector('#resultListDiv > table > tbody > tr') :
                    results.append(td.text.split())

                if len(results) > 2 :
                    for i in range (1, len(results)-1) :
                            pre_df_team_h.append({
                                "연도" : target_year,
                                "월"   : target_month,
                                "일"   : target_day,
                                "팀명" : results[i][1],
                                'R/G' : results[i][8], 
                                '득점' : results[i][9], 
                                '홈런' : results[i][10], 
                                '도루' : results[i][11], 
                                '볼넷%' : results[i][12], 
                                '삼진%' : results[i][13], 
                                'BABIP' : results[i][14], 
                                '타율' : results[i][15], 
                                '출루율' : results[i][16], 
                                '장타율' : results[i][17], 
                                'OPS' : results[i][18],
                                'wOBA' : results[i][19]
                            }) 

            except Exception as e:
                print('오류 발생', target_year, target_month, target_day, e)

In [ ]:
hitDf = pd.DataFrame(pre_df_team_h)
hitDf

,연도,월,일,팀명,R/G,득점,홈런,도루,볼넷%,삼진%,BABIP,타율,출루율,장타율,OPS,wOBA
0,2020,07,01,두산,14.00,14,1,0,6.7,8.9,0.486,0.452,0.489,0.595,1.084,0.483
1,2020,07,01,KT,11.00,11,3,0,2.2,13.3,0.457,0.432,0.444,0.727,1.171,0.504
2,2020,07,01,KIA,4.00,4,0,0,5.0,20.0,0.407,0.314,0.385,0.343,0.728,0.335
3,2020,07,01,삼성,7.00,7,0,1,10.5,21.0,0.400,0.303,0.395,0.364,0.759,0.357
4,2020,07,01,NC,6.00,6,2,1,11.8,8.8,0.167,0.207,0.303,0.448,0.751,0.328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,2020,10,24,LG,3.00,3,1,0,0.0,17.4,0.216,0.200,0.196,0.311,0.507,0.219
906,2020,10,25,KIA,10.00,10,3,1,14.0,11.6,0.345,0.351,0.442,0.622,1.064,0.453
907,2020,10,25,KT,10.00,10,1,1,9.3,9.3,0.367,0.343,0.425,0.543,0.968,0.423
908,2020,10,25,롯데,5.00,5,2,0,10.0,2.5,0.212,0.250,0.325,0.472,0.797,0.348


In [ ]:
hitDf.to_csv('kbreport_hit_main_day_200701_201025.csv', encoding='cp949')